# Aplicar funciones a elementos de `Series` o `DataFrame`s
Cualquier procesado, manipulación o análisis de conjuntos de datos implica el aplicar transformaciones, funciones o calcular indicadores elemento a elemento, fila a fila o columna a columna. 

In [ ]:
# Preliminares

Para ilustrar, generamos un `DataFrame` usando un generador de números aleatorios.

In [ ]:
# Usamos el generador por defecto del módulo random de numpy
# 

## Funciones universales `ufuncs` de `numpy`
Puesto que un `DataFrame` también es un array de `numpy`, podemos aplicarle cualquier función universal de `numpy`. Las funciones universales de `numpy`(ufuncs) se pueden consultar en la referencia [Available ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html#available-ufuncs)

In [ ]:
# Calculamos la exponencial de x e y, elemento a elemento

In [ ]:
# Elevamos x a la potencia y, elemento a elemento:

## También se pueden aplicar métodos de `pandas` directamente a todo un `DataFrame`

In [ ]:
# Para obtener la suma, columna por columna

In [ ]:
# Para obtener la media, columna por columna

In [ ]:
# Para obtener la desviación típica, columna por columna

También podríamos calcular estos indicadores, fila por fila, usando el parámetro `axis=1`.

Podéis encontrar y explorar la gran cantidad de métodos disponibles para un `DataFrame`en la [documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) de `pandas`.

## Aplicar funciones a elementos de `Series` o `DataFrame`s (II)
En el vídeo anterior vimos cómo aplicar funciones universales de `numpy` o los métodos de `pandas` a `DataFrame`s. Pero puede ser que queramos aplicar nuestras propias funciones. 

Al igual que en el vídeo anterior, para ilustrar, generamos un `DataFrame` usando un generador de números aleatorios.

In [ ]:
# Usamos el generador por defecto del módulo random de numpy

Empezamos por crear una función

In [ ]:
# hemos usado la construcción if else

Para aplicar nuestra función a cada elemento del `DataFrame` usamos `applymap`

Si quisieramos aplicar nuestra función a cada elemento de una `Serie` usaríamos `apply`:

## Las funciones anónimas en Python

Es muy útil tener la posibilidad de definir sobre la marcha una función sin darle un nombre y usarla como argumento de un método. Se hace con `lambda`.

In [ ]:
# Para obtener el doble de cada elemento

## El método `apply` para un `DataFrame`

El método `apply` si se aplica a un `DataFrame` debe tener como argumento a una función que se aplica a toda una columna o toda una fila, no a un elemento individual.

In [ ]:
# Ejemplo con una función que se aplica a un vector: calculamos el número de valores positivos por columna

El método `apply` admite también el argumento `axis=1`, para que se aplique la función fila por fila.

> Es muy recomendable evitar `apply` si es posible, y usar las funciones de `numpy` o los métodos de `pandas` que son muy optimizados y llevan a cabo la iteración en C.
`apply` lleva a cabo la iteración en Python.

En el caso anterior, podríamos haber usado directamente el método `sum` en `pandas`.

In [ ]:
# Fila por fila: